In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../dataset/retail.csv")
df.sample(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID
397558,580682,23483,HANGING BUTTERFLY T-LIGHT HOLDER,12,05/12/2011 15:02,1.25,16968
138651,553838,22755,SMALL PURPLE BABUSHKA NOTEBOOK,24,19/05/2011 12:10,0.85,12620
46504,542503,84971S,SMALL HEART FLOWERS HOOK,24,28/01/2011 11:58,0.85,16327
5122,537036,22625,RED KITCHEN SCALES,1,03/12/2010 17:28,8.50,15545
27182,540023,22670,FRENCH WC SIGN BLUE METAL,1,04/01/2011 12:58,1.25,15039
81975,546769,22423,REGENCY CAKESTAND 3 TIER,7,16/03/2011 14:57,12.75,17504
7956,537241,22111,SCOTTIE DOG HOT WATER BOTTLE,9,06/12/2010 10:18,4.95,14293
244133,566300,21524,DOORMAT SPOTTY HOME SWEET HOME,4,11/09/2011 16:05,8.25,18215
50841,543040,21890,S/6 WOODEN SKITTLES IN COTTON BAG,2,02/02/2011 17:06,2.95,17337
282724,569910,23284,DOORMAT KEEP CALM AND COME IN,10,06/10/2011 16:47,7.08,17139


In [3]:
df = df.loc[df["Quantity"] > 0]

Check for nan values

In [20]:
df.to_csv("../dataset/clean-retail.csv", index=False)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 397924 entries, 0 to 406828
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    397924 non-null  object 
 1   StockCode    397924 non-null  object 
 2   Description  397924 non-null  object 
 3   Quantity     397924 non-null  int64  
 4   InvoiceDate  397924 non-null  object 
 5   UnitPrice    397924 non-null  float64
 6   CustomerID   397924 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 24.3+ MB


In [5]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01/12/2010 08:26,2.55,17850
1,536365,71053,WHITE METAL LANTERN,6,01/12/2010 08:26,3.39,17850
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01/12/2010 08:26,2.75,17850
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01/12/2010 08:26,3.39,17850
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01/12/2010 08:26,3.39,17850


In [6]:
customer_item_matrix = df.pivot_table(
    index='CustomerID',
    columns='StockCode',
    values='Quantity',
    aggfunc='sum'
)
print(customer_item_matrix.shape)

customer_item_matrix = customer_item_matrix.applymap(lambda x: 1 if x > 0 else 0)

(4339, 3665)


## User Based Colloborative filtering 

In [7]:
from sklearn.metrics.pairwise import cosine_similarity

user_to_user_smatrix = pd.DataFrame(cosine_similarity(customer_item_matrix))
user_to_user_smatrix.head()

,0,1,2,3,4,5,6,7,8,9,...,4329,4330,4331,4332,4333,4334,4335,4336,4337,4338
0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1,0.0,1.000000,0.063022,0.046130,0.047795,0.038484,0.0,0.025876,0.136641,0.094742,...,0.0,0.029709,0.052668,0.0,0.032844,0.062318,0.0,0.113776,0.109364,0.012828
2,0.0,0.063022,1.000000,0.024953,0.051709,0.027756,0.0,0.027995,0.118262,0.146427,...,0.0,0.064282,0.113961,0.0,0.000000,0.000000,0.0,0.000000,0.170905,0.083269
3,0.0,0.046130,0.024953,1.000000,0.056773,0.137137,0.0,0.030737,0.032461,0.144692,...,0.0,0.105868,0.000000,0.0,0.039014,0.000000,0.0,0.067574,0.137124,0.030475
4,0.0,0.047795,0.051709,0.056773,1.000000,0.031575,0.0,0.000000,0.000000,0.033315,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.044866,0.000000


In [8]:
#Renaming index and column names

user_to_user_smatrix.columns = customer_item_matrix.index

user_to_user_smatrix['CustomerID'] = customer_item_matrix.index
user_to_user_smatrix = user_to_user_smatrix.set_index('CustomerID')
user_to_user_smatrix.head()

CustomerID,12346,12347,12348,12349,12350,12352,12353,12354,12355,12356,...,18273,18274,18276,18277,18278,18280,18281,18282,18283,18287
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
12347,0.0,1.000000,0.063022,0.046130,0.047795,0.038484,0.0,0.025876,0.136641,0.094742,...,0.0,0.029709,0.052668,0.0,0.032844,0.062318,0.0,0.113776,0.109364,0.012828
12348,0.0,0.063022,1.000000,0.024953,0.051709,0.027756,0.0,0.027995,0.118262,0.146427,...,0.0,0.064282,0.113961,0.0,0.000000,0.000000,0.0,0.000000,0.170905,0.083269
12349,0.0,0.046130,0.024953,1.000000,0.056773,0.137137,0.0,0.030737,0.032461,0.144692,...,0.0,0.105868,0.000000,0.0,0.039014,0.000000,0.0,0.067574,0.137124,0.030475
12350,0.0,0.047795,0.051709,0.056773,1.000000,0.031575,0.0,0.000000,0.000000,0.033315,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.044866,0.000000


Top 10 Similar users to user id **12350**

In [9]:
user_to_user_smatrix.loc[12350.0].sort_values(ascending=False).head(10)

CustomerID
12350    1.000000
17935    0.183340
12414    0.181902
12652    0.175035
16692    0.171499
12791    0.171499
16754    0.171499
12814    0.171499
16426    0.166968
16333    0.161690
Name: 12350, dtype: float64

### Making recommendations for user B (id : 17935) 

In [25]:
user_to_user_smatrix.loc[12350.0].sort_values(ascending=False).index[1]

17935

In [10]:
user_to_user_smatrix.loc[12350.0].sort_values(ascending=False)
products_bought_by_A = customer_item_matrix.loc[12350.0][customer_item_matrix.loc[12350.0]>0]
print("Products Bought by A: ")
print(products_bought_by_A)

Products Bought by A: 
StockCode
20615     1
20652     1
21171     1
21832     1
21864     1
21866     1
21908     1
21915     1
22348     1
22412     1
22551     1
22557     1
22620     1
79066K    1
79191C    1
84086C    1
POST      1
Name: 12350, dtype: int64


In [11]:
products_bought_by_B = customer_item_matrix.loc[17935.0][customer_item_matrix.loc[17935.0]>0]
print("Products bought by B:")
print(products_bought_by_B)

Products bought by B:
StockCode
20657     1
20659     1
20828     1
20856     1
21051     1
21866     1
21867     1
22208     1
22209     1
22210     1
22211     1
22449     1
22450     1
22551     1
22553     1
22557     1
22640     1
22659     1
22749     1
22752     1
22753     1
22754     1
22755     1
23290     1
23292     1
23309     1
85099B    1
POST      1
Name: 17935, dtype: int64


In [12]:
products_to_recommend_to_B = set(products_bought_by_A.index) - set(products_bought_by_B.index)
print("Product Recommendation for User B ")
df.loc[df['StockCode'].isin(products_to_recommend_to_B),['StockCode', 'Description']].drop_duplicates().set_index('StockCode')

Product Recommendation for User B 


,Description
StockCode,
21832,CHOCOLATE CALCULATOR
21915,RED HARMONICA IN BOX
22620,4 TRADITIONAL SPINNING TOPS
79066K,RETRO MOD TRAY
21864,UNION JACK FLAG PASSPORT COVER
79191C,RETRO PLASTIC ELEPHANT TRAY
21908,CHOCOLATE THIS WAY METAL SIGN
20615,BLUE POLKADOT PASSPORT COVER
20652,BLUE POLKADOT LUGGAGE TAG


## Item based collaborative filtering

In [13]:
product_similarity = pd.DataFrame(cosine_similarity(customer_item_matrix.T))
product_similarity.columns = customer_item_matrix.T.index

product_similarity['StockCode'] = customer_item_matrix.T.index
product_similarity = product_similarity.set_index('StockCode')

In [14]:
product_similarity

StockCode,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,...,90214V,90214W,90214Y,90214Z,BANK CHARGES,C2,DOT,M,PADS,POST
StockCode,,,,,,,,,,,,,,,,,,,,,
10002,1.000000,0.000000,0.094868,0.091287,0.0,0.000000,0.090351,0.062932,0.098907,0.095346,...,0.000000,0.0,0.000000,0.0,0.000000,0.029361,0.0,0.066915,0.0,0.078217
10080,0.000000,1.000000,0.000000,0.000000,0.0,0.000000,0.032774,0.045655,0.047836,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.016182,0.0,0.000000
10120,0.094868,0.000000,1.000000,0.115470,0.0,0.000000,0.057143,0.059702,0.041703,0.060302,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.070535,0.0,0.010993
10123C,0.091287,0.000000,0.115470,1.000000,0.0,0.000000,0.164957,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000
10124A,0.000000,0.000000,0.000000,0.000000,1.0,0.447214,0.063888,0.044499,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C2,0.029361,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.036955,0.019360,0.055989,...,0.000000,0.0,0.000000,0.0,0.000000,1.000000,0.0,0.026196,0.0,0.020413
DOT,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.104257,0.150756,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,1.0,0.000000,0.0,0.000000
M,0.066915,0.016182,0.070535,0.000000,0.0,0.000000,0.070535,0.070185,0.066184,0.106335,...,0.049875,0.0,0.040723,0.0,0.089220,0.026196,0.0,1.000000,0.0,0.077539


In [15]:
top_10_similar_products = list(product_similarity.loc['23166'].sort_values(ascending=False).iloc[:10].index)

### Similar Product Recommendation to Product 23166

In [16]:
df.loc[
    df['StockCode'].isin(top_10_similar_products),
    ['StockCode', 'Description']
].drop_duplicates().set_index('StockCode').loc[top_10_similar_products]

,Description
StockCode,
23166,MEDIUM CERAMIC TOP STORAGE JAR
23165,LARGE CERAMIC TOP STORAGE JAR
23167,SMALL CERAMIC TOP STORAGE JAR
22993,SET OF 4 PANTRY JELLY MOULDS
23307,SET OF 60 PANTRY DESIGN CAKE CASES
22722,SET OF 6 SPICE TINS PANTRY DESIGN
22720,SET OF 3 CAKE TINS PANTRY DESIGN
22666,RECIPE BOX PANTRY YELLOW DESIGN
23243,SET OF TEA COFFEE SUGAR TINS PANTRY


In [21]:
import pickle 

with open("../models/product_based_rec.pickle", "wb") as f:
    pickle.dump(product_similarity, f, protocol=pickle.HIGHEST_PROTOCOL)

with open("../models/user_based_rec.pickle", "wb") as f:
    pickle.dump(user_to_user_smatrix, f, protocol=pickle.HIGHEST_PROTOCOL)

with open("../models/user_product_matrix.pickle", "wb") as f:
    pickle.dump(customer_item_matrix, f, protocol=pickle.HIGHEST_PROTOCOL)